In [3]:
!gcloud ml-engine jobs submit training linear_estimator_`date +%s` \
--staging-bucket gs://linear-estimator/ \
--module-name trainer.task \
--package-path trainer \
--region us-central1 \
--runtime-version 1.8 \
--python-version 2.7

Job [linear_estimator_1542032819] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe linear_estimator_1542032819

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs linear_estimator_1542032819
jobId: linear_estimator_1542032819
state: QUEUED


In [4]:
!gcloud ml-engine models create linear_estimator

Created ml engine model [projects/devlabs-208303/models/linear_estimator].


In [7]:
!gcloud ml-engine versions create v1 \
--model linear_estimator \
--origin gs://linear-estimator/train/export/linear/1542032923 \
--runtime-version=1.8

Creating version (this might take a few minutes)......done.                     


In [9]:
from googleapiclient.discovery import build
from oauth2client.client import GoogleCredentials
from datetime import datetime
import numpy as np

samples = np.array([8., 9.])

credentials = GoogleCredentials.get_application_default()
service = build('ml', 'v1', credentials=credentials)
request = service.projects().predict(
              name='projects/devlabs-208303/models/linear_estimator/versions/v1',
              body={"instances": samples.tolist()
                   })
response = request.execute()
print(response)

{u'predictions': [{u'predictions': [15.46474552154541]}, {u'predictions': [17.457897186279297]}]}


In [10]:
predictions = response['predictions']

In [15]:
for input, p in zip(samples, predictions):
    v  = p["predictions"][0]
    print("{0} -> {1:.4f}".format(input, v))

8.0 -> 15.4647
9.0 -> 17.4579


In [16]:
!gcloud ml-engine predict \
--model=linear_estimator --version=v1 \
--json-instances=./f1.json

PREDICTIONS
[15.46474552154541]
[17.457897186279297]


In [18]:
!gcloud ml-engine jobs submit prediction linear_estimator_pred_`date +%s` \
--region us-central1 \
--data-format=TEXT \
--input-paths=gs://linear-estimator/batchpred/f1.json \
--output-path=gs://linear-estimator/batchpred/outputs \
--model=linear_estimator --version=v1

Job [linear_estimator_pred_1542035187] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe linear_estimator_pred_1542035187

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs linear_estimator_pred_1542035187
jobId: linear_estimator_pred_1542035187
state: QUEUED
